# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 连接数据库

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT tprm.order_id, om.order_number, om.create_time 下单日期, om.status 订单状态, tprm.relet_periods 续租期数, om.order_finish_date 订单完成时间, tod.sku_attributes,
tod.new_actual_money 买断价, tprm.all_deposit 总押金, tprm.total_receivable 总应收, tprm.total_received 总已收, om.all_money 总租金, (tod.new_actual_money-om.all_money) 买断尾款1, tprm.purchase_amount 采购金额, om.has_actual 是否买断, tprm.rembursement_status 还款状态, om.relet_status 是否续租, tprm.paid_periods 已还期数, tod.early_completion_mark 是否提前完成, tmu.id_card_num 身份证号, tol.delivery_province_name 地区, pa.type 活动类型, pa.type, too.money 滞纳金, tprm.liquidated_damages 违约金
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.order_method, om.merchant_name
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
left join db_digua_business.t_platform_activity pa on om.activity_id=pa.id
left join db_digua_business.t_member_user tmu on om.user_id=tmu.id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order_logistics tol on tol.order_id=tprm.order_id
left join (SELECT t.order_id,sum(case when t.capital_type =6 and t.`status`=2 then t.money end) as other_money 
,sum(case when t.capital_type != 5 AND  t.`status` = 2 then t.money end) as money
from db_digua_business.t_order_other t GROUP BY 1 ) too on too.order_id=om.id
;
'''

df_order = query(sql1)
df = df_order.copy()
df = df[df.type!=4]
df.shape

(34227, 29)

In [5]:
sql_rent = '''
select ymos.order_id,  ymos.refund_date 应还日期, ymos.reality_refund_date 实还日期, ymos.status 支付状态, ymos.sort 期数, ymos.money 应还租金, ymos.part_payment 实收租金
,too.pay_date_new  买断月份, too.sum_money 应付买断金, too.sum_real_pay_money 实付买断金, ymos.order_pay_id 还款支付记录, ymos.is_relet, ymos.money
-- , case when sort=1 then date_format(refund_date, '%Y-%m') else 'nan' end 放款月份
from db_rent.ya_merchant_order_stages ymos
left join (select order_id, date_format(pay_date, '%Y-%m') pay_date_new, sum(money) sum_money, sum(real_pay_money) sum_real_pay_money from db_digua_business.t_order_out where  status!=1 group by order_id, date_format(pay_date, '%Y-%m')) too
on too.order_id=ymos.order_id and too.pay_date_new=date_format(ymos.refund_date, '%Y-%m')
'''
df_rent = query(sql_rent)
df_rent.shape
# pay_status=2 and

(1362334, 13)

# 历史渠道补充

In [6]:
qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])

# 处理数据

In [7]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a

df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df = clean.drop_merchant(df)
df2 = df.copy()
df = clean.drop_rejected_merchant(df2)

In [8]:
df_tmp = pd.read_excel('F:/租后表分析数据_ld/tmp11(1).xlsx')
df = df_tmp.merge(df, on='order_number', how='left')

In [9]:
df_tmp.shape

(31686, 4)

In [10]:
df.shape

(31689, 36)

In [11]:
# df = df[(df.归属渠道=='芝麻租物')|(df.归属渠道=='搜索渠道')]#
df.loc[:, '下单日期'] = df.下单日期.dt.date
df.loc[:, '下单月份'] = df.下单日期.dt.strftime('%Y-%m')
df.loc[:, '下单年份'] = pd.to_datetime(df.search_time).dt.year
df.loc[:, '滞纳金'] = df.滞纳金.fillna(0)
df.loc[:, '违约金'] = df.违约金.fillna(0)
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
# 当为租完即送和续租的订单，买断尾款为0
# df.loc[:, '买断尾款'] = np.where((df.租赁方案=='租完即送')|(df.是否续租==2), 0, df.买断尾款)
df.loc[:, '实收买断尾款'] = np.where(df.是否买断==1, df.买断尾款, 0)


# df.loc[:, '提前结清金额'] = np.where((df.是否提前完成==2)|(df.是否买断==1), df.买断尾款+df.滞纳金+df.违约金, 0)
# 分组统计
def group(df, name, year):
    df_group = df.groupby(name).agg({'应付总租金': 'sum', '买断尾款': 'sum', 'order_number': 'nunique'}).rename(columns={'应付总租金': f'{year}至今总租金', '买断尾款': f'{year}至今买断尾款', 'order_number': f'{year}至今客户数'})
    return df_group
# 整理地区
df['地区'] = np.where(df['地区']=='上海', '上海市', 
                    np.where(df['地区']=='北京', '北京市',
                            np.where(df['地区']=='天津', '天津市',
                                np.where(df['地区']=='重庆', '重庆市', df['地区']))))
df.loc[:, '地区'] = np.where(df.地区.isna(), '未知地区', df.地区)
# 划分金额区间
df.loc[:, '采购金额'] = np.where(df.采购金额.isna(), 0, df.采购金额)
money_bins = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, float('inf')]
money_labels = ['[0,1000)', '[1000,2000)', '[2000,3000)', '[3000,4000)', '[4000,5000)', '[5000,6000)', '[6000, 7000)',  '[7000, 8000)',  '[8000, 9000)',  '[9000, 10000)', '[10000+)']

# 使用 pd.cut 设置金额区间
df['money_group'] = pd.cut(df['采购金额'], bins=money_bins, labels=money_labels, right=False, include_lowest=True)

# 划分时间
df_2021 = df[df.search_time>='2021-01']
df_2022 = df[df.search_time>='2022-01']
df_2023 = df[df.search_time>='2023-01']
df_2024 = df[df.search_time>='2024-01']
df_2025 = df[df.search_time>='2025-01']

# 客户来源

In [12]:
## 2021年至今 
df_2021_group = group(df_2021, '下单年份', '2021').cumsum().iloc[-1:, :]
## 2022年至今
df_2022_group = group(df_2022, '下单年份', '2022').cumsum().iloc[-1:, :]
## 2023年至今
df_2023_group = group(df_2023, '下单年份', '2023').cumsum().iloc[-1:, :]
## 2024年至今
df_2024_group = group(df_2024, '下单年份', '2024').cumsum().iloc[-1:, :]
## 2025年至今
df_2025_group = group(df_2025, '下单年份', '2025').cumsum().iloc[-1:, :]
df_customer_source = pd.concat([df_2021_group, df_2022_group, df_2023_group, df_2024_group, df_2025_group], axis=1)
df_customer_source

,2021至今总租金,2021至今买断尾款,2021至今客户数,2022至今总租金,2022至今买断尾款,2022至今客户数,2023至今总租金,2023至今买断尾款,2023至今客户数,2024至今总租金,2024至今买断尾款,2024至今客户数,2025至今总租金,2025至今买断尾款,2025至今客户数
下单年份,,,,,,,,,,,,,,,
2025,1.985151e+08,1.589706e+08,31686,1.985151e+08,1.589706e+08,31686,1.969168e+08,1.579244e+08,31414,1.418691e+08,1.163854e+08,22433,12202579.94,9645316.46,1863


# 客户地区

In [13]:
# df_2021[['order_number', '地区']].to_excel('F:/客户地区.xlsx', index=False)

In [14]:
# group(df_2023, '地区', '2023').cumsum()

In [15]:
## 2021至今
df_2021_p_group = group(df_2021, '地区', '2021')
## 2022至今
df_2022_p_group = group(df_2022, '地区', '2022')
## 2023至今
df_2023_p_group = group(df_2023, '地区', '2023')
## 2024至今
df_2024_p_group = group(df_2024, '地区', '2024')
## 2025至今
df_2025_p_group = group(df_2025, '地区', '2025')
df_customer_address = pd.concat([df_2021_p_group, df_2022_p_group, df_2023_p_group, df_2024_p_group, df_2025_p_group], axis=1)
df_customer_address

,2021至今总租金,2021至今买断尾款,2021至今客户数,2022至今总租金,2022至今买断尾款,2022至今客户数,2023至今总租金,2023至今买断尾款,2023至今客户数,2024至今总租金,2024至今买断尾款,2024至今客户数,2025至今总租金,2025至今买断尾款,2025至今客户数
地区,,,,,,,,,,,,,,,
上海市,5952772.27,4737425.53,936,5952772.27,4737425.53,936,5901153.17,4698442.95,928,4167653.38,3393440.72,648,256181.73,222900.47,37.0
云南省,6294593.32,5031155.57,1013,6294593.32,5031155.57,1013,6254006.12,4993292.77,1002,4292137.91,3553596.71,680,346238.74,243460.36,48.0
内蒙古自治区,3202120.13,2540190.54,496,3202120.13,2540190.54,496,3189251.13,2540190.54,495,2403015.05,1959054.05,367,209080.82,191218.78,32.0
北京市,4771370.36,3769075.54,746,4771370.36,3769075.54,746,4738540.01,3745100.89,741,3666478.73,2963984.35,571,367609.99,270061.41,52.0
吉林省,3939199.22,3136748.78,607,3939199.22,3136748.78,607,3917212.02,3120258.98,604,3084474.88,2499088.02,470,260078.96,216015.04,38.0
四川省,11249548.89,8735285.98,1764,11249548.89,8735285.98,1764,11204101.85,8693729.82,1752,7436946.62,5962258.98,1162,634488.67,499503.83,90.0
天津市,2121371.75,1625049.53,326,2121371.75,1625049.53,326,2087300.76,1598546.52,321,1465736.47,1166744.41,227,188284.73,148980.27,27.0
宁夏回族自治区,914663.41,692620.47,144,914663.41,692620.47,144,903899.41,683886.47,142,601842.95,494365.63,96,57201.51,41572.89,8.0
安徽省,9073093.13,7130310.04,1409,9073093.13,7130310.04,1409,9034871.11,7105885.38,1402,6895393.50,5573438.34,1067,655905.37,493372.33,92.0


# 金额分布

In [16]:
def money_rate(df, df_all, year):
    # df_all.to_numpy()[0, -1] 取到固定行和列的值
    df.loc[:, '金额占比'] = ((df[f'{year}至今总租金']+df[f'{year}至今买断尾款'])/(df_all.to_numpy()[0, 0]+df_all.to_numpy()[0, 1])).map(lambda x: format(x, '.2%'))
    df.loc[:, '客户数占比'] = (df[f'{year}至今客户数']/df_all.to_numpy()[0, 2]).map(lambda x: format(x, '.2%'))
    df_new = df[[f'{year}至今客户数', '客户数占比', f'{year}至今总租金', f'{year}至今买断尾款', '金额占比']]
    return df_new

In [17]:
# money_rate(group(df_2022, 'money_group', '2022'), df_2022_group, '2022').cumsum()

In [18]:
# 2021至今
df_2021_m_group = money_rate(group(df_2021, 'money_group', '2021'), df_2021_group, '2021')
# 2022至今
df_2022_m_group = money_rate(group(df_2022, 'money_group', '2022'), df_2022_group, '2022')
# 2023至今
df_2023_m_group = money_rate(group(df_2023, 'money_group', '2023'), df_2023_group, '2023')
# 2024至今
df_2024_m_group = money_rate(group(df_2024, 'money_group', '2024'), df_2024_group, '2024')
# 2025至今
df_2025_m_group = money_rate(group(df_2025, 'money_group', '2025'), df_2025_group, '2025')
df_money = pd.concat([df_2021_m_group, df_2022_m_group, df_2023_m_group, df_2024_m_group, df_2025_m_group], axis=1)
df_money

,2021至今客户数,客户数占比,2021至今总租金,2021至今买断尾款,金额占比,2022至今客户数,客户数占比,2022至今总租金,2022至今买断尾款,金额占比,2023至今客户数,客户数占比,2023至今总租金,2023至今买断尾款,金额占比,2024至今客户数,客户数占比,2024至今总租金,2024至今买断尾款,金额占比,2025至今客户数,客户数占比,2025至今总租金,2025至今买断尾款,金额占比
money_group,,,,,,,,,,,,,,,,,,,,,,,,,
"[0,1000)",925,2.92%,4637918.22,3356349.39,2.24%,925,2.92%,4637918.22,3356349.39,2.24%,923,2.94%,4624858.90,3344971.71,2.25%,830,3.70%,4242050.25,3056159.35,2.83%,159,8.53%,510066.59,226885.11,3.37%
"[1000,2000)",350,1.10%,573313.51,261749.48,0.23%,350,1.10%,573313.51,261749.48,0.23%,336,1.07%,552391.69,249065.10,0.23%,191,0.85%,317771.55,125351.35,0.17%,31,1.66%,64206.00,5515.00,0.32%
"[2000,3000)",393,1.24%,987208.64,512381.31,0.42%,393,1.24%,987208.64,512381.31,0.42%,384,1.22%,965046.60,502116.31,0.41%,191,0.85%,475354.30,251094.70,0.28%,3,0.16%,9324.00,1764.00,0.05%
"[3000,4000)",600,1.89%,2021653.61,1138216.58,0.88%,600,1.89%,2021653.61,1138216.58,0.88%,580,1.85%,1955335.82,1100231.69,0.86%,342,1.52%,1155565.59,598352.01,0.68%,5,0.27%,18047.62,11499.28,0.14%
"[4000,5000)",915,2.89%,3863863.54,2703317.31,1.84%,915,2.89%,3863863.54,2703317.31,1.84%,903,2.87%,3811797.46,2668491.71,1.83%,579,2.58%,2506518.39,1674680.61,1.62%,14,0.75%,57404.03,43890.67,0.46%
"[5000,6000)",2016,6.36%,9423794.46,7209162.90,4.65%,2016,6.36%,9423794.46,7209162.90,4.65%,1973,6.28%,9231207.74,7063352.86,4.59%,942,4.20%,4765161.78,3112855.12,3.05%,43,2.31%,220493.57,143946.73,1.67%
"[6000, 7000)",3233,10.20%,15813141.85,16033641.57,8.91%,3233,10.20%,15813141.85,16033641.57,8.91%,3221,10.25%,15747868.65,15992716.77,8.95%,2177,9.70%,10488702.92,10996406.88,8.32%,30,1.61%,165578.42,105846.78,1.24%
"[7000, 8000)",6219,19.63%,36473444.82,33791227.75,19.66%,6219,19.63%,36473444.82,33791227.75,19.66%,6199,19.73%,36348667.10,33699350.47,19.74%,4430,19.75%,26159339.02,24189789.45,19.50%,45,2.42%,271708.14,220370.76,2.25%
"[8000, 9000)",10583,33.40%,76163165.44,57417037.33,37.37%,10583,33.40%,76163165.44,57417037.33,37.37%,10531,33.52%,75812519.32,57166259.15,37.48%,7872,35.09%,56119304.38,43672010.34,38.64%,931,49.97%,6543849.26,5363993.14,54.50%


# 还款计划

In [19]:
df_rent_group = df_rent.groupby('order_id').agg(期数总租金=('money', 'sum'))

In [20]:
# 获取已支付的期数
df_rent.loc[:, '当前期数'] = df_rent[(df_rent.支付状态==3)].groupby('order_id').期数.transform('max')
# 用前一个非空值来填充当前缺失值
df_rent.loc[:, '当前期数'] = df_rent.当前期数.ffill()
df_rent.loc[:, '还款月份'] = df_rent.应还日期.dt.strftime('%Y-%m')
df_rent.loc[:, 'max_期数'] = df_rent.groupby('order_id').期数.transform('max')
df_rent.loc[:, 'max_未续租期数'] = df_rent[df_rent.is_relet==0].groupby('order_id').期数.transform('max')
df_rent.loc[:, 'max_未续租期数'] = df_rent['max_未续租期数'].ffill()
df_rent.loc[:, 'max_续租期数'] = df_rent[df_rent.is_relet>=1].groupby('order_id').期数.transform('max')
df_rent.loc[:, 'max_续租期数'] = df_rent['max_续租期数'].bfill()
df_rent.loc[:, '续租期数'] = df_rent['max_续租期数']-df_rent['max_未续租期数']
df_rent.loc[:, '续租期数'] = df_rent['续租期数'].fillna(0)
df_rent = df_rent.merge(df_rent_group, on='order_id', how='left')
# 对表进行关联
df_merge = df.merge(df_rent, on='order_id', how='left')
# df_merge.loc[:, 'max_期数'] = df_merge.groupby('order_id').期数.transform('max')
df_merge = df_merge[(df_merge.max_未续租期数==12)|(df_merge.续租期数_y==12)]
# 获取应还时间和订单完成时间
df_merge['date_tmp_1'] = pd.to_datetime(df_merge['应还日期'].dt.strftime('%Y-%m'))
df_merge['date_tmp_2'] = pd.to_datetime(np.where(df_merge['订单完成时间'].notnull(), pd.to_datetime(df_merge['订单完成时间']).dt.strftime('%Y-%m'), np.nan))
# 判断订单结束日期非空的订单是买断还是非买断，未结束的订单不判断（资方所需的是是否结清，分买断或完成）
df_merge['订单状态_new'] = df_merge.apply(lambda x: '已买断' if x.date_tmp_1 == x.date_tmp_2 and x.还款状态 == '已买断'
                                                else '已完成' if x.date_tmp_1 == x.date_tmp_2 and x.还款状态 == '已完成'
                                                else '还款中' if x.date_tmp_1 < x.date_tmp_2
                                                else '已结束' if x.date_tmp_1 > x.date_tmp_2
                                                else '其他', axis = 1
                                                )
df_merge.loc[:, '当前期数'] = np.where(df_merge.还款支付记录=='order_pay', df_merge.期数, df_merge.当前期数)
df_merge.loc[:, '当前期数'] = df_merge.groupby('order_id').当前期数.transform('min')
df_merge.loc[:, '提前结清金额'] = np.where((df_merge.订单状态_new=='已买断')|((df_merge.订单状态_new=='已完成')&(df_merge.当前期数<df_merge.max_期数)), df_merge.实付买断金+df_merge.滞纳金+df_merge.违约金, 0)
df_merge.loc[:, '应还租金'] = np.where((df_merge.当前期数>=df_merge.期数), df_merge.应还租金, 0)
df_merge.loc[:, '买断尾款'] = np.where((df_merge.当前期数>=df_merge.期数), df_merge.买断尾款, 0)
df_merge.loc[:, '实收租金'] = np.where((df_merge.当前期数>=df_merge.期数), df_merge.实收租金, 0)
df_merge.loc[:, '实收买断尾款'] = np.where((df_merge.当前期数>=df_merge.期数), df_merge.实收买断尾款, 0)
# 分组获取数据
df_merge1 = df_merge[df_merge.期数==df_merge.max_期数]
df_merge_group = df_merge1.groupby('下单月份').agg({'order_id': 'nunique', '期数总租金': 'sum', '买断尾款': 'sum', '总押金': 'sum'}).rename(columns={'order_id': '客户数'})
df_merge_group_sort = df_merge.groupby(['下单月份', '还款月份', '期数']).agg({'应还租金': 'sum', '应付买断金': 'sum', '实收租金': 'sum', '实付买断金': 'sum', '提前结清金额': 'sum'}).rename(columns={'买断尾款': '应还买断尾款'})
df_merge_group_sort = df_merge_group_sort.reset_index().set_index('下单月份')
# 关联获取的数据
df_merge_play = df_merge_group.merge(df_merge_group_sort, on='下单月份', how='left')
df_merge_play.to_excel('F:/ts.xlsx')

In [21]:
df_merge_play

,客户数,期数总租金,买断尾款,总押金,还款月份,期数,应还租金,应付买断金,实收租金,实付买断金,提前结清金额
下单月份,,,,,,,,,,,
2022-07,22,156993.13,53762.74,149233.0,2022-07,1.0,7522.10,0.0,7522.10,0.0,0.0
2022-07,22,156993.13,53762.74,149233.0,2022-08,2.0,10818.57,0.0,9605.96,0.0,0.0
2022-07,22,156993.13,53762.74,149233.0,2022-09,3.0,10818.57,0.0,5917.11,0.0,0.0
2022-07,22,156993.13,53762.74,149233.0,2022-10,4.0,10818.57,0.0,5917.11,0.0,0.0
2022-07,22,156993.13,53762.74,149233.0,2022-11,5.0,10818.57,0.0,5917.11,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2025-01,1735,11905686.34,8752085.50,16938835.9,2025-08,8.0,895547.50,0.0,1156.11,0.0,0.0
2025-01,1735,11905686.34,8752085.50,16938835.9,2025-09,9.0,895547.52,0.0,1156.11,0.0,0.0
2025-01,1735,11905686.34,8752085.50,16938835.9,2025-10,10.0,895547.80,0.0,9021.13,0.0,0.0


# 异常租赁场景（提前买断+提前退租）

In [22]:
# 判断是否是提前买断和是否提前结清
df_merge = df_merge1.copy()
# 提前买断：已买断+期数<12或期数为12但存在续租期数
df_merge.loc[:, '提前买断客户数'] = np.where((df_merge.还款状态=='已买断')&((df_merge.已还期数<12)|((df_merge.已还期数==12)&(df_merge.续租期数_x>0))), 1, 0)
df_merge.loc[:, '提前结清客户数'] = np.where((df_merge.提前买断客户数==1)|(df_merge.当前期数<df_merge.max_期数)&(df_merge.订单状态==8)&(df_merge.订单状态_new=='已结束'), 1, 0)
# 将不是提前买断和不是提前结清的期数填充为1
# df_merge.loc[:, '提前买断客户数'] = df_merge.提前买断客户数.replace(0, np.nan).ffill()
# df_merge.loc[:, '提前结清客户数'] = df_merge.提前结清客户数.replace(0, np.nan).ffill()
# 提前买断和提前结清已收金额
df_merge.loc[:, '提前买断已收金额'] = np.where(df_merge.提前买断客户数==1, df_merge.总已收, 0)
df_merge.loc[:, '提前结清已收金额'] = np.where(df_merge.提前结清客户数==1, df_merge.总已收, 0)

In [23]:
# 进行分组统计
df_merge_tq = df_merge.groupby('下单月份').agg({'order_id': 'nunique', '买断价': 'sum'}).rename(columns={'order_id': '客户数'})
# 对提前买断的客户进行统计
df_merge_tq1 = df_merge[df_merge.提前买断客户数==1]
df_merge_tq1_group = df_merge_tq1.groupby('下单月份').agg({'order_id': 'nunique', '提前买断已收金额': 'sum'}).rename(columns={'order_id': '提前买断客户数'})
# 对提前结清的客户进行统计（包含提前买断的）
df_merge_tq2 = df_merge[df_merge.提前结清客户数==1]
df_merge_tq2_group = df_merge_tq2.groupby('下单月份').agg({'order_id': 'nunique', '提前结清已收金额': 'sum'}).rename(columns={'order_id': '提前结清客户数'})
df_merge_tq_all = pd.concat([df_merge_tq, df_merge_tq1_group, df_merge_tq2_group], axis=1)
df_merge_tq_all

,客户数,买断价,提前买断客户数,提前买断已收金额,提前结清客户数,提前结清已收金额
下单月份,,,,,,
2022-07,22,199705.91,4,38043.36,5,38614.97
2022-08,56,537792.73,13,132124.24,13,132124.24
2022-09,44,412112.72,14,141333.54,14,141333.54
2022-10,41,347869.90,12,103291.89,13,115668.04
2022-11,43,454886.06,15,141675.85,16,155709.75
2022-12,66,758505.46,21,237417.36,22,244316.96
2023-01,221,2223392.80,65,653247.37,65,653247.37
2023-02,332,3330099.70,77,826675.12,80,859778.33
2023-03,316,3197303.85,75,815834.07,77,833200.90


# 还款计划

In [24]:
# df = df.merge(df_rent, on='order_id', how='left')
# df = df[(df.max_未续租期数==12)|(df.续租期数==12)]
# df2 = df2.merge(df_rent, on='order_id', how='left')
# df2 = df2[(df2.max_未续租期数==12)|(df2.续租期数==12)]

# df2_ck = df2[df2.是否出库==1]
# df2_ck_group = df2_ck.groupby('下单月份').agg({'purchase_amount': 'sum', 'new_actual_money': 'sum', 'all_money': 'sum'})
# df2_ck_group.loc[:, '买断尾款'] = df2_ck_group.new_actual_money-df2_ck_group.all_money
# df2_ck_group.rename(columns={'purchase_amount': '采购金额', 'new_actual_money': '买断价', 'all_money': '总租金'}, inplace=True)
# df_all[['进件数', '机审通过件', '出库（按下单时间）']].merge(df2_ck_group, on='下单月份', how='left')